# EuroLeague 2015-16 Season: Offensive Rebounding

Combined file for easier analysis

389 players listed, 353 got playing time (had a pbp event involving them)

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.patches import Wedge
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection


import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [2]:
import csv
from tabulate import tabulate
import numpy as np
from scipy.spatial import distance
from collections import Counter
from pprint import pprint
from IPython.display import Image

In [3]:
f = open('pbp.csv', 'r')
reader = csv.DictReader(f, delimiter=',', quotechar='"')
pbp=[]
for line in reader:
    pbp.append(line)
f.close()

In [4]:
f = open('roster.csv', 'r')
reader = csv.DictReader(f, delimiter=',', quotechar='"')
roster=[]
for line in reader:
    roster.append(line)
f.close()

## Housekeeping / Basic Stats
(prefer roster dictionary for overlapping data)

In [5]:
gamecodes=[]
for r in roster:
    if r['gamecode'] not in gamecodes:
        gamecodes.append(r['gamecode'])
len(gamecodes)

250

In [6]:
### DICTIONARY: KEY=abbrev VALUE=fullname
### e.g. team_names['ULK'] = 'Fenerbahce Istanbul'
team_names={}
for r in pbp:
    if r['team_id'] not in team_names:
        team_names[r['team_id']] = r['team_name']
    else:
        if team_names[r['team_id']] != r['team_name']:
            print('ERROR')
len(team_names)

25

In [7]:
team_ids = []
for key in team_names:
    if key != '':
        team_ids.append(key)
len(team_ids)

24

In [8]:
### player_infos: KEY=player_id; VALUE={dict}
###                                 ['name'] = [all name variations] 
###                                 ['team'] = [multiple teams if traded mid-season]
###                                 ['events'] = [all relevant pbps]

### e.g. player_infos['P005983']['name'] == ['YURTSEVEN, OMER FARUK', 'YURTSEVEN, OMER']

player_infos={}

for r in roster:
    if 'CO_' in r['player_id'] or r['player_id'] == '' or r['player_name'] == '': continue
    if r['player_id'] not in player_infos:
        player_infos[r['player_id']] = {}
        player_infos[r['player_id']]['seconds'] = 0
        player_infos[r['player_id']]['gamecodes_starter'] = {}
        player_infos[r['player_id']]['gamecodes_bench'] = {}
        player_infos[r['player_id']]['events'] = []
        player_infos[r['player_id']]['team'] = []
        player_infos[r['player_id']]['team'].append(r['team_id'])
        player_infos[r['player_id']]['name'] = [] #multiple names
        player_infos[r['player_id']]['name'].append(r['player_name'])

    else:
        if r['team_id'] not in player_infos[r['player_id']]['team']:
            print('dupe team for', r['player_id'], 'add', r['team_id'], '->', player_infos[r['player_id']]['team'])
            player_infos[r['player_id']]['team'].append(r['team_id'])
        temp_name=r['player_name'].strip()
        if temp_name not in player_infos[r['player_id']]['name']:
            print('dupe name for', r['player_id'], 'add', temp_name, '->', player_infos[r['player_id']]['name'])
            player_infos[r['player_id']]['name'].append(temp_name)
    
    #Additional feature would be to separate games/games_started_vs._benched/minutes per team for players with multiple teams in the same season, but left unresolved for simplicity. 
    if r['starter_flag'] == '1':
        if float(r['mp_as_int']) != 0:
            player_infos[r['player_id']]['gamecodes_starter'][r['gamecode']] = float(r['mp_as_int'])
    elif r['starter_flag'] == '0':
        if float(r['mp_as_int']) != 0:
            player_infos[r['player_id']]['gamecodes_bench'][r['gamecode']] = float(r['mp_as_int'])
    else: print('invalid starter_flag')
    player_infos[r['player_id']]['seconds'] += float(r['mp_as_int'])
    
for r in pbp:
    if 'CO_' in r['player_id'] or r['player_id'] == '' or r['player_name'] == '': continue
    if r['team_id'] not in player_infos[r['player_id']]['team']:
        print('pbp dupe team for', r['player_id'], 'add', r['team_id'], '->', player_infos[r['player_id']]['team'])
        player_infos[r['player_id']]['team'].append(r['team_id'])
    temp_name=r['player_name'].strip()
    if temp_name not in player_infos[r['player_id']]['name']:
        print('pbp dupe name for', r['player_id'], 'add', temp_name, '->', player_infos[r['player_id']]['name'])
        player_infos[r['player_id']]['name'].append(temp_name)
    player_infos[r['player_id']]['events'].append(r)

dupe team for PKJP add TEL -> ['RED']
dupe name for P005983 add YURTSEVEN, OMER -> ['YURTSEVEN, OMER FARUK']
dupe name for P006648 add BAYKAN, OGULCAN -> ['BAYCAN OGULCAN']
dupe name for P004264 add BOST, DEE -> ['BOST, DEMARQUIS']
dupe name for P001389 add VORONOV, EVGENY -> ['VORONOV, EUGENY']
dupe name for P005367 add UTA, CHISTIAN -> ['UTA, RARES']
dupe team for P002580 add RED -> ['MUN']
dupe team for P002429 add MAD -> ['MUN']
dupe team for P003601 add PAN -> ['SAS']
dupe name for P007016 add JOHNSON, ELIJAH -> ['JOHNSON, ELIJAH KWAME']


In [9]:
### special case handling for a player with 2 player_ids

yurtseven={}
yurtseven['seconds'] = player_infos['P005983']['seconds'] + player_infos['P005353']['seconds']
yurtseven['gamecodes_starter'] = {**player_infos['P005983']['gamecodes_starter'],
                                          **player_infos['P005353']['gamecodes_starter']}
yurtseven['gamecodes_bench'] = {**player_infos['P005983']['gamecodes_bench'],
                                        **player_infos['P005353']['gamecodes_bench']}
yurtseven['name'] = list(set(player_infos['P005983']['name'] + player_infos['P005353']['name']))
yurtseven['team'] = list(set(player_infos['P005983']['team'] + player_infos['P005353']['team']))

yurtseven['events'] = player_infos['P005983']['events'] + player_infos['P005353']['events']
player_infos['P005983'] = yurtseven
player_infos['P005353'] = yurtseven

In [10]:
for key, p in player_infos.items():
    p['gamecodes_all'] = {**p['gamecodes_starter'], **p['gamecodes_bench']}

5 more player names than ids discrepancy is explained by one player having more than one name

In [11]:
event_desc=[]
for r in pbp:
    if r['event_desc'] not in event_desc:
        event_desc.append(r['event_desc'])
len(event_desc)

3114

In [12]:
event_desc_id=[]
for r in pbp:
    if r['event_desc_id'] not in event_desc_id:
        event_desc_id.append(r['event_desc_id'])
len(event_desc_id)


29

In [13]:
### games organized by gamecode
### KEY=string gamecode; VALUE=[team1, team2] 

games_gamecode={}
for r in roster:
    if r['gamecode'] not in games_gamecode.keys():
        games_gamecode[r['gamecode']]=[]
    if r['team_id'] not in games_gamecode[r['gamecode']]:
        games_gamecode[r['gamecode']].append(r['team_id'])
        if len(games_gamecode[r['gamecode']]) > 2:
            print('ERROR:', str(r['gamecode']), r)

games_gamecode['1'], games_gamecode['250']

(['KHI', 'MAD'], ['CSK', 'ULK'])

In [14]:
### games organized by team
games_team_id={}
for t in team_ids:
    games_team_id[t]=[]
for key, g in games_gamecode.items():
    for t in team_ids:
        if g[0] == t or g[1] == t:
            games_team_id[t].append((key, g))
            
games_team_id['TEL']

[('59', ['DAR', 'TEL']),
 ('82', ['MAL', 'TEL']),
 ('72', ['CSK', 'TEL']),
 ('12', ['CSK', 'TEL']),
 ('107', ['BAM', 'TEL']),
 ('96', ['SAS', 'TEL']),
 ('119', ['DAR', 'TEL']),
 ('47', ['BAM', 'TEL']),
 ('36', ['SAS', 'TEL']),
 ('22', ['MAL', 'TEL'])]

# Analysis

## Identify the top 5 teams & players in offensive rebounding. Describe the metric(s) used and explain why you used it.

### offensive rebounding percentage (orebs / missed shots)
(drawing loose ball fouls after own team's missed shots)

was a player in play when a team missed a shot? (we can find out based on substitutions)

In [15]:
### TOTAL (OREBS / MISSED) SHOTS PER TEAM
orebs_player = {}
orebs_team = {}
missedshots_team = {}
madeshots_team = {}
for r in pbp:
    if r['event_desc_id'] == 'O':
        if r['player_id'] not in orebs_player.keys():
            orebs_player[r['player_id']] = []
        orebs_player[r['player_id']].append(r)
        if r['team_id'] not in orebs_team.keys():
            orebs_team[r['team_id']] = []
        orebs_team[r['team_id']].append(r)
    elif any([r['event_desc_id'] == e for e in ('2FGA', '2FGAB', '3FGA', '3FGAB')]):
        #consider last missed free throw
        if r['team_id'] not in missedshots_team.keys():
            missedshots_team[r['team_id']] = []
        missedshots_team[r['team_id']].append(r)
    elif any([r['event_desc_id'] == e for e in ('2FGM', '3FGM')]):
        if r['team_id'] not in madeshots_team.keys():
            madeshots_team[r['team_id']] = []
        madeshots_team[r['team_id']].append(r)


In [16]:
### orebs, missed_shots, orebs/missed_shots, missed_shots per game
temp=[]
for team_id in team_ids:
    temp.append((team_id,
                 len(games_team_id[team_id]),
                 len(orebs_team[team_id]),
                 len(missedshots_team[team_id]),
                 round(len(orebs_team[team_id]) / len(missedshots_team[team_id]), 4),
                 round(len(orebs_team[team_id]) / len(games_team_id[team_id]), 2),
                 round(len(missedshots_team[team_id]) / len(games_team_id[team_id]), 2),
                 round(len(madeshots_team[team_id]) / len(games_team_id[team_id]), 2),
                 round(len(madeshots_team[team_id]) /
                        (len(madeshots_team[team_id]) + len(missedshots_team[team_id])), 4)))
    
### calculate league averages
### code below weighs each team totals equally, even if they have different number of games/shots/orebs/etc
# temp_avg=[]
# for i, column in enumerate(zip(temp)):
#     if i == 0: temp_avg.append('AVG')
#     elif i < 4: temp_avg.append('')
# for i, each in enumerate(reversed(np.rot90([t[4:] for t in temp]))):
#     if i == 0 or i == 4: temp_avg.append(round(np.mean(each), 4))
#     else: temp_avg.append(round(np.mean(each), 2))

print(tabulate(reversed(sorted(temp, key=lambda item: item[4])), headers=('', 'games', 'orebs', '- shots', 'oreb%', 'orebs/game', '- shots/game', '+ shots/game', 'fg%')))
# print()
# print(tabulate((temp_avg, []), headers=('', 'games', 'orebs', '- shots', 'oreb%', 'orebs/game', '- shots/game', '+ shots/game', 'fg%')))

       games    orebs    - shots    oreb%    orebs/game    - shots/game    + shots/game     fg%
---  -------  -------  ---------  -------  ------------  --------------  --------------  ------
MAL       24      313        821   0.3812         13.04           34.21           26.54  0.4369
IST       24      276        791   0.3489         11.5            32.96           29.96  0.4762
TEL       10      123        356   0.3455         12.3            35.6            26.6   0.4277
OLY       24      283        836   0.3385         11.79           34.83           27.33  0.4397
MAD       27      312        942   0.3312         11.56           34.89           29.19  0.4555
CSK       29      277        843   0.3286          9.55           29.07           31.17  0.5175
ULK       29      303        925   0.3276         10.45           31.9            28.21  0.4693
KHI       24      272        831   0.3273         11.33           34.62           29.96  0.4639
RED       27      282        864   0.326

## Top 5 Offensive Rebounding Teams

The top 7 teams in offensive rebounding rate (oreb%) are discussed below. The preceding numbers represent a team's rank in oreb%.

1) __Unicaja Malaga [MAL]__'s numbers are eye-popping; the team's oreb% is the league-high at an absurd 38%, with the next best team falling below 35%. With this statistic alone, _one can comfortably conclude that __MAL is at least one of the top 5 offensive rebounding teams, if not the best.__

6) __CSKA Moscow [CSK]__ is an interesting team. Their oreb% is a respectable 33%, which falls to 6th best in the league. However, they are incredibly efficient on offense. They boast league-best numbers with most shots made and least shots missed per game (which leads to highest field goal percentage (fg%)). It's one thing for a team to jack up a bunch of mediocre shots and get lots of long offensive rebounds (4 of 5 top oreb% teams are at or below league average fg%, while the next 14 teams have above average fg%), but CSK prioritizes offensive shot quality while still maintaining a great oreb%. If CSK were to punt shot quality in favor of offensive rebounds, I would expect their oreb% to surge upward. Despite being outside of the top 5 in oreb%, __I would consider CSK to be one of the 5 best offensive rebound teams.__

3) __~~Maccabi FOX Tel Aviv [TEL]~~__ is the opposite of CSK. Although their oreb% is 34.5%, good for 3rd in the league, they are the only team among the top 14 oreb% teams not to have advanced to the playoffs. In addition, they have the 5th lowest fg% in the league. For TEL, their high oreb% might actually be a side-effect of their awful shot quality rather than their knack for offensive rebounding, as discussed above. If their shot quality improves, their oreb% likely goes down. Not only that, there are only 10 games of data for TEL, containing 123 offensive rebounds from 356 missed shots; such a sample size is too big to ignore altogether, but it is small enough to be susceptible to statistical noise. __In a vacuum, I would not consider TEL to be one of the 5 best offensive rebounding teams.__

2) __Anadolu Efes Istanbul [IST]__ is like CSK, but more well-rounded. Their oreb% is 35%, 2nd best in the league; 48% fg% is 3rd best. Compared to CSK, IST gets on the offensive glass more, while their shot quality is lower. In a vacuum, I would say IST's offensive rebounding prowess is equal to or slightly higher than CSK's. __To me, IST is in the top 5.__

4) __Olympiacos Piraeus [OLY]__ and 5) __~~Real Madrid [MAD]~~__ are very similar in many ways. They get miss shots and get offensive rebounds at about the same rate per game, while MAD makes two more on two more attempts per game. In short, MAD has better shot quality, whereas OLY crashes the offensive glass more. They are probably very close in terms of offensive rebounding as a team, but __I will take OLY based the oreb% numbers.__

7) __Fenerbahce Istanbul [ULK]__ __slips in as my 5th best offensive rebounding team__. In the same vein as CSK and IST, ULK seems to prioritize shot quality, with their fg% at 4th best. However, there is a sizable group of teams that hover around 32.5 and 33.5 oreb%, all of which could be considered for the 5th spot.

### My list for top 5 teams in offensive rebounding in descending order:
1. Unicaja Malaga [MAL]
2. Anadolu Efes Istanbul [IST]
3. CSKA Moscow [CSK]
4. Olympiacos Piraeus [OLY]
5. Fenerbahce Istanbul [ULK]

Note: The given data has some minor discrepancies (in total oreb count) from the stats page on EuroLeague.net; likely data collection errors, but small enough to ignore.

### Offensive rebound after missed FT (only 191 in 250 games). This is already counted in the general orebs table below, so kind of pointless

In [17]:
oreb_after_fta={}
for t in team_ids:
    oreb_after_fta[t]=[]
for i, r in enumerate(pbp):
    if r['event_desc_id'] == 'FTA' and pbp[i+1]['event_desc_id'] == 'O':
        for t in team_ids:
            if t == pbp[i+1]['team_id']:
                oreb_after_fta[t].append(pbp[i+1])
temp = []
for t in team_ids:
    temp.append((t, round(len(oreb_after_fta[t]) / len(games_team_id[t]), 4), len(oreb_after_fta[t])))
    
sorted(temp, key=lambda item: item[1])

[('MUN', 0.1, 1),
 ('MIL', 0.1, 1),
 ('TEL', 0.1, 1),
 ('STR', 0.2, 2),
 ('KSK', 0.2, 2),
 ('LMG', 0.2, 2),
 ('DAR', 0.2083, 5),
 ('TIV', 0.2258, 7),
 ('BAS', 0.2759, 8),
 ('ULK', 0.2759, 8),
 ('BAM', 0.2917, 7),
 ('ZAL', 0.2917, 7),
 ('CSK', 0.3448, 10),
 ('CED', 0.375, 9),
 ('GSS', 0.4, 4),
 ('MAL', 0.4167, 10),
 ('KHI', 0.4167, 10),
 ('BAR', 0.4483, 13),
 ('RED', 0.4815, 13),
 ('SAS', 0.5, 5),
 ('IST', 0.5417, 13),
 ('OLY', 0.6667, 16),
 ('MAD', 0.6667, 18),
 ('PAN', 0.7037, 19)]

In [18]:
### EVENT IMMEDIATELY FOLLOWING MISSED FREE THROW
events_after_fta={}
for i, r in enumerate(pbp):
    if r['event_desc_id'] == 'FTA':
        if pbp[i+1]['event_desc_id'] not in events_after_fta.keys():
            events_after_fta[pbp[i+1]['event_desc_id']] = []
        events_after_fta[pbp[i+1]['event_desc_id']].append((i, pbp[i+1]))

temp=[]
for e in events_after_fta.keys():
    temp.append((e, len(events_after_fta[e])))
        
sorted(temp, key=lambda item: item[1])

[('TOUT', 1),
 ('ST', 1),
 ('3FGM', 2),
 ('RV', 4),
 ('2FGA', 9),
 ('TO', 10),
 ('3FGA', 12),
 ('2FGM', 12),
 ('IN', 14),
 ('CM', 14),
 ('AS', 33),
 ('O', 191),
 ('FTA', 311),
 ('FTM', 823),
 ('D', 886)]

# best oreb player:

## Top 5 Offensive Rebounding Players

### My top 5 in descending order

1. __HUNTER, OTHELLO [P005160]__
2. __KUZMIC, OGNJEN [P003479]__
3. __AYON, GUSTAVO [P005927]__
4. __REYES, FELIPE [PAAX]__
5. __ZIRBES, MAIK [P004183]__


__AYON, GUSTAVO [P005927]__ is the league leader in total offensive rebounds (orebs) and orebs per game (orebs/g), though with a lot of games and minutes per game. But even when adjusted for such, he ranks in the top 5 for orebs per 40 minutes (orebs/40m). He is on the fringe of the elite class of offensive rebounders (4.5+ orebs/40m), but his consistency and reliability on the offensive glass gives him an edge. When adjusted for the defensive rebounding rate of his opponents, his oreb numbers on top by a large margin. His raw numbers seem to underrate him, but I think __belongs in the top 5__.

On the other hand, __KUZMIC, OGNJEN [P003479]__ has a lot of potential to be a special offensive rebounder. His league-leading 5.8 orebs/40m is off the charts. He only gets 13 minutes a game, which means his numbers are more susceptible to being fluky, it's all about gambling on upside and potential for prospects from outside the US. In the EuroLeague, I would expect him to dominate as he gets more minutes, __so he is in__.

__~~STIMAC, VLADIMIR [PLBJ]~~__ is very similar to Kuzmic, in fact grabbing the same 1.92 orebs/g with one more minute per game than Kuzmic. He is in the elite class of offensive rebounders, but when adjusted for defensive rebounding rate of his opponents, his numbers fall flat. He also showed troubling trends towards the end of the season, steadily losing orebs, and putting up multiple 0-oreb games in the playoffs. __He's not in my top 5__.

__HUNTER, OTHELLO [P005160]__ is 2nd best in orebs/40m as well as orebs/g, and he has more games, minutes, and total orebs than Kuzmic. His stats say he is very productive and well-rounded; he even logs starter minutes while still upholding his fantastic 5.43 orebs/40m, so __he is my number 1 pick__.

__REYES, FELIPE [PAAX]__'s numbers may seem a bit shaky, and he seems to land somewhere near the top 5. But among the leading offensive rebounders in the EuroLeague, he was one of the only players to show a steady growth in offensive rebounding through the season.

__ZIRBES, MAIK [P004183]__ is Ayon-lite. His offensive rebounding production falls short of Ayon by all metrics, but his consistency is not to be ignored. He puts up starter minutes, with decent per minute numbers, as well as against very strong opposition. __He just barely makes the cut__.

Stats used: orebs, minutes per game, adjusted orebs/40m: when a player grabs an offensive rebound, its value depends on the opponent's defensive rebounding prowess. In the EuroLeague, there is a large gap between the best and worst defensive rebounding teams. Therefore, the raw orebs/40m is adjusted based on opponent defensive rebounding percentage (e.g. an oreb against a team with a dreb% of 80% is worth 0.8 adjusted oreb, instead of a flat 1).



### Drawing defensive loose ball fouls is too noisy, too small, useless.
* look at drawing loose ball fouls after team missed shot
* fouls are not descriptive, so after FGA -> foul drawn without Oreb or Dreb in between

In [ ]:
temp=[]
for i, r in enumerate(pbp):
    if any([r['event_desc_id'] == e for e in ('2FGA', '3FGA', 'FTA', '2FGAB', '3FGAB')]):
        temp_team = r['team_id']
        k = i
        while True:
            i+=1
            if pbp[i]['event_desc_id'] == 'RV' and pbp[i]['team_id'] == temp_team:
                temp.append((k, i))
                break
            if any([pbp[i]['event_desc_id'] == e for e in ('FTA', 'FTM', 'O', 'D', 'BP')]):
                break

the data is inconsistent when it comes to loose ball fouls after a missed shot. generally, it's
1. #FGA(B)/FTA missed shot
 * AG shot rejected
 * FV block
2. RV foul drawn
3. CM foul
4. (O offensive rebound)

Sometimes it skips the 

In [ ]:
### num of events from a friendly missed shot to a foul drawn
onf=Counter([t[1]-t[0] for t in temp])
pprint(onf)

In [ ]:
onf[1]

In [ ]:
onf12 = [pbp[t[1]]['player_name'] for t in temp if t[1]-t[0] < 3]

list(reversed(sorted(list(Counter(onf12).items()), key=lambda x: x[1])))

## Time of season for oreb numbers, look for trends

In [ ]:
for r in roster:
    print(r)
    break

In [19]:
def opponent(gamecode, player_id):
    teams = player_infos[player_id]['team']
    for t in games_gamecode[gamecode]:
        for team in teams:
            if t != team:
                return t
    return None

temp=[]
temp_headers = ('player_id', 'player_name', 'orebs', 'minutes', 'games', 'm/g', 'orebs/game', 'orebs/40m', 'adj o/40m')
for player_id, r in player_infos.items():
    temp_orebs = [e for e in r['events'] if e['event_desc_id'] == 'O']
    adj_orebs=0
    for t in temp_orebs:
        adj_orebs += round((oreb_multiplier_against_team[opponent(t['gamecode'], player_id)]) / 40, 4)
    temp_games = len(r['gamecodes_starter']) + len(r['gamecodes_bench'])
    if len(temp_orebs) == 0 or temp_games == 0: continue #handles 'divide by zero' exceptions
    if temp_games < 15 or len(temp_orebs) < 40: continue
    
    temp.append((player_id,
                 r['name'][0],
                 len(temp_orebs),
                 round((r['seconds'] / 60), 2),
                 temp_games,
                 round((r['seconds'] / 60) / (temp_games), 2),
                 round(len(temp_orebs) / (temp_games), 2),
                 round(60 * 40 * len(temp_orebs) / r['seconds'], 2),
                 adj_orebs))
    
print(tabulate(list(reversed(sorted(temp, key=lambda x:x[6]))), headers=temp_headers))

NameError: name 'oreb_multiplier_against_team' is not defined

can we determine large gamecodes are later games?

In [ ]:
team_names['CSK']

highest gamecodes: last few games are the championship games, 3rd place games, and late playoff games, so seems likely

In [ ]:
for i in range(250, 220, -1):
    print(i, team_names[games_gamecode[str(i)][0]], '|', team_names[games_gamecode[str(i)][1]])

In [ ]:
roster[0]

here first games of the regular season are listed at the lowest gamecodes.

In [ ]:
for i in range(1, 20):
    print(i, team_names[games_gamecode[str(i)][0]], '|', team_names[games_gamecode[str(i)][1]])

It actually goes by Rounds and Groups; e.g. R1GA, R1GB, R1GC, R1GD, R2G1...
Last games of Group A are often played before the first games of Group B in the same round, which means the smaller numbers do not strictly mean earlier games in the season. (i.e. Round 1 Group A Game 3 (gamecode 3), is played after Round 1 Group B Game 1 (gamecode 4))

But going by rounds (which are 1 week apart), it's generally safe to say:

### gamecode ~= ordinal game date.
This means we can look at trends for players' oreb numbers as the season goes along.

In [ ]:
### LEARN NUMPY

candidates=['P003479',
            'P005160',
            'PLBJ',
            'PAAX',
            'P005927',
            'P004183']
temp={}
for c in candidates:
    temp[c] = sorted(list(Counter([int(r['gamecode']) for r in orebs_player[c]]).items()), key=lambda x: x[0])

In [ ]:
for i in range(1, 251):
    for c in candidates:
        if str(i) in player_infos[c]['gamecodes_all']:
            if i not in [t[0] for t in temp[c]]:
                temp[c].append((i, 0))
        temp[c]=sorted(temp[c], key=lambda x:x[0])

In [ ]:
temp['PLBJ']

A scatterplot is used to see if there are any visible trends in the candidates' oreb numbers per game.

In [ ]:
for c in candidates:
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111)
    ax.scatter([t[0] for t in temp[c]], [t[1] * 60 * 20 / player_infos[c]['gamecodes_all'][str(t[0])] for t in temp[c]], marker='+', color='k')
    #ax.scatter([t[0] for t in temp[c]], [t[1] for t in temp[c]], [60000 / player_infos[c]['gamecodes_all'][str(t[0])] for t in temp[c]])
    #ax.scatter([t[0] for t in temp[c]], [t[1] for t in temp[c]], s = 50, c = 'r', )
    ax.set_xlim(1, 250)
    ax.set_ylim(0, 15)
    ax.set_xlabel('gamecode')
    ax.set_ylabel('oreb / 20m')
    ax.set_title(c + ' '+ str(player_infos[c]['name']))
    plt.show()

## oreb player update: Reyes is in, Stimac is out

* Stimac has great numbers, but is inconsistent/trending down. He also put up 4 games with 0 orebs towards the end of the season, a concerning sign.
* Also Stimac has a fluke game where he got 5:30 minutes of gameplay and got 3 orebs, which raised his orebs/40m stats.
* Meanwhile, Reyes is getting better towards the end of the season.

In [ ]:
for g, mp in player_infos['PLBJ']['gamecodes_all'].items():
    if int(g) > 150 and int(g) < 200:
        print(g, mp)

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
c = 'PLBJ'
stimac = ax.scatter([t[0] for t in temp[c]], [t[1] * 60 * 20 / player_infos[c]['gamecodes_all'][str(t[0])] for t in temp[c]], facecolors='r')
c = 'PAAX'
reyes = ax.scatter([t[0] for t in temp[c]], [t[1] * 60 * 20 / player_infos[c]['gamecodes_all'][str(t[0])] for t in temp[c]], facecolors='k')
c = 'P003479'
#kuzmic = ax.scattber([t[0] for t in temp[c]], [t[1] * 60 * 20 / player_infos[c]['gamecodes_all'][str(t[0])] for t in temp[c]], edgecolors='b', facecolors = 'w', s=80)
ax.set_xlabel('gamecode')
ax.set_ylabel('oreb / 20m')


plt.legend((stimac, reyes), ('Stimac, Vladimir', 'Reyes, Felipe'))
#, 'Kuzmic, Ognjen'

plt.show()

players' offensive rebounding numbers % defensive rebounding %

needs defensive 5 using IN and OUT

### trends for teams

In [ ]:
candidates=['MAL', 'CSK', 'TEL', 'IST', 'OLY', 'MAD', 'ULK', 'KHI']

#pprint([(team, games) for team, games in games_team_id.items() if team in candidates])
temp={}
for c in candidates:
    temp[c] = []
    gc_o=sorted([(gamecode, oreb_count) for gamecode, oreb_count in Counter([int(o['gamecode']) for o in orebs_team[c]]).items()], key=lambda x: x[0])
    gc_m=[x[1] for x in sorted([(gamecode, missedshots_count) for gamecode, missedshots_count in Counter([int(o['gamecode']) for o in missedshots_team[c]]).items()], key=lambda x: x[0])]
    for i, o in enumerate(gc_o):
        temp[c].append((gc_o[i][0], round((gc_o[i][1] / gc_m[i]), 2)))
        
temp
    

MAD is the only team with a significant downward trend

KHI is almost there, but raw numbers are not good enough

In [ ]:
candidates=['MAD', 'ULK', 'KHI', 'IST', 'OLY']
for c in candidates:
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111)
    ax.scatter([t[0] for t in temp[c]], [t[1] for t in temp[c]])
    ax.set_xlim(1, 250)
    ax.set_ylim(0, .6)
    ax.set_xlabel('gamecode')
    ax.set_ylabel('oreb%')
    ax.set_title(c)
    plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
temp_candidates= ['MAD'] #'IST', 'OLY', 'MAD', 'ULK', 'KHI'
colors = ('k', 'r', 'b', 'g')
scats=[]
for i, c in enumerate(temp_candidates):
    scats.append(ax.scatter([t[0] for t in temp[c]], [t[1] for t in temp[c]], color=colors[i]))
    #ax.plot([t[0] for t in temp[c]], [t[1] for t in temp[c]], color=colors[i], linestyle='-')
ax.set_xlim(1, 250)
ax.set_ylim(0, 0.6)
ax.set_xlabel('gamecode')
ax.set_ylabel('oreb%')
ax.set_title(c)

plt.legend(scats, temp_candidates)
plt.show()

## Teams: Defensive Rebounding

In [ ]:
i=0
for r in pbp:
    if any([r['event_desc_id'] == e for e in('2FGA', '2FGAB', '3FGA', '3FGAB')]): i += 1
i

In [ ]:
missedshots_against_team = {}
dreb_team = {}
oreb_multiplier_against_team = {}
for t in team_ids:
    missedshots_against_team[t] = []
    dreb_team[t] = []
    oreb_multiplier_against_team[t] = 0
    
for r in pbp:
    if any([r['event_desc_id'] == e for e in('2FGA', '2FGAB', '3FGA', '3FGAB')]):
        for t in games_gamecode[r['gamecode']]:
            if t != r['team_id']:
                defense = t
        missedshots_against_team[defense].append(r)
    if r['event_desc_id'] == 'D':
        dreb_team[r['team_id']].append(r)

temp=[]   
for t in team_ids:
    temp.append((t, len(dreb_team[t]), len(missedshots_against_team[t]), round(len(dreb_team[t]) / len(missedshots_against_team[t]), 4)))
    oreb_multiplier_against_team[t] = len(dreb_team[t]) / len(missedshots_against_team[t])
    

    
print(tabulate(reversed(sorted(temp, key=lambda x: x[3])), headers=('team', 'drebs', 'opp - shots', 'dreb%')))



In [ ]:
games_gamecode['1']

### Basic Stats

In [ ]:
### OVERALL FG%
fgm = sum([len(m) for team, m in madeshots_team.items()])
fga = sum([len(m) for team, m in missedshots_team.items()])
print('fg% {} {}/{}'.format((round(fgm / (fgm + fga), 4)), fgm, (fgm+fga)))

temp=[]
for team in madeshots_team:
    for shot in madeshots_team[team]:
        if shot['event_desc_id'] == '2FGM':
            temp.append(shot)
fgm2_team=temp            
fgm2 = len(temp)

temp=[]
for team in missedshots_team:
    for shot in missedshots_team[team]:
        if shot['event_desc_id'] == '2FGA' or shot['event_desc_id'] == '2FGAB':
            temp.append(shot)
fga2_team=temp            
fga2 = len(temp)

print('2fg% {} {}/{}'.format((round(fgm2 / (fgm2 + fga2), 4)), fgm2, (fgm2+fga2)))

temp=[]
for team in madeshots_team:
    for shot in madeshots_team[team]:
        if shot['event_desc_id'] == '3FGM':
            temp.append(shot)
fgm3_team=temp            
fgm3 = len(temp)

temp=[]
for team in missedshots_team:
    for shot in missedshots_team[team]:
        if shot['event_desc_id'] == '3FGA' or shot['event_desc_id'] == '3FGAB':
            temp.append(shot)
fga3_team=temp        
fga3 = len(temp)

print('3fg% {} {}/{}'.format((round(fgm3 / (fgm3 + fga3), 4)), fgm3, (fgm3+fga3)))

###eFG% = (2FGM + 1.5*3FGM) / (FGM + FGA)
print('efg% {} ({} + 1.5 * {}) / ({} + {})'.format(round(((fgm2 + 1.5 * fgm3)/(fgm + fga)), 4), fgm2, fgm3, fgm, fga))

In [ ]:
### FG% AFTER OREB
#define shot after oreb:
#     oreb by TeamA
#     -----------------
#     TeamA does not TO, OF, FV, D, ST
#     TeamB does not 2FGA, 2FGAB, 2FGM, 3FGA, 3FGAB, 3FGM, AG, O, TO, OF, FTA, FTM
#     There are no BP, EG 
#     There are no play stoppages IN, OUT, TOUT_TV, TOUT, CMU, CMT, B, CMD
#     -----------------
#     fgm/fga by TeamA
temp=[]
oreb_fgs = [] #{'oreb', 'shot'}
for i, r in enumerate(pbp):
    if r['event_desc_id'] == 'O':
        temp_oreb_fgs = {}
        temp_oreb_fgs['oreb'] = r
        k=i
        offense = r['team_id']
        while True:
            i += 1
            if i - k < 5 and pbp[i]['team_id'] == offense and any([pbp[i]['event_desc_id'] == e for e in ('2FGA', '2FGAB', '2FGM', '3FGA', '3FGAB', '3FGM')]):
                temp.append(pbp[i])
                temp_oreb_fgs['shot'] = pbp[i]
                oreb_fgs.append(temp_oreb_fgs)
                break
            if pbp[i]['team_id'] == offense and any([pbp[i]['event_desc_id'] == e for e in ('O', 'TO', 'OF', 'FV', 'D', 'ST')]):
                break
            if pbp[i]['team_id'] != offense and any([pbp[i]['event_desc_id'] == e for e in ('2FGA', '2FGAB', '2FGM', '3FGA', '3FGAB', '3FGM', 'AG', 'O', 'TO', 'OF', 'FTA', 'FTM', 'BP', 'EG')]):
                break
            if any([pbp[i]['event_desc_id'] == e for e in ('IN', 'OUT', 'TOUT_TV', 'TOUT', 'CMU', 'CMT', 'B', 'CMD')]):
                break
                    
oreb_fg = temp

play stoppage means the offensive rebounding team reverts to a half-court offense, which negates the post-oreb possession. therefore, it is removed from consideration for fg% after orebs.

In [ ]:
fgm2_split_team = {}
fga2_split_team = {}
fgm3_split_team = {}
fga3_split_team = {}

for team in team_ids:
    fgm2_split_team[team] = []
    fga2_split_team[team] = []
    fgm3_split_team[team] = []
    fga3_split_team[team] = []
    
for f in fgm2_team:
    fgm2_split_team[f['team_id']].append(f)
for f in fga2_team:
    fga2_split_team[f['team_id']].append(f)
for f in fgm3_team:
    fgm3_split_team[f['team_id']].append(f)
for f in fga3_team:
    fga3_split_team[f['team_id']].append(f)
    
    

In [ ]:
temp_2chance = {}
temp_overall = {}
for team in team_ids:
    temp_overall[team] = {}
    temp_overall[team]['events'] = []
    temp_2chance[team] = {}
    temp_2chance[team]['events'] = []
for o in oreb_fgs:
    temp_2chance[o['oreb']['team_id']]['events'].append(o)
    
# print(temp_2chance['CSK']['events'][0]['shot'])
   
for team in team_ids:
    temp_2chance[team]['2fgm']=[e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '2FGM']
    temp_2chance[team]['2fga']=[e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '2FGA'] + [e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '2FGAB']
    temp_2chance[team]['3fgm']=[e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '3FGM']
    temp_2chance[team]['3fga']=[e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '3FGA'] + [e for e in temp_2chance[team]['events'] if e['shot']['event_desc_id'] == '3FGAB']

temp_2change_efg = []
for team in team_ids:
    temp_2change_efg.append([team,
                             len(temp_2chance[team]['events']),
                             len(temp_2chance[team]['2fgm']),
                             len(temp_2chance[team]['2fga']),
                             len(temp_2chance[team]['3fgm']),
                             len(temp_2chance[team]['3fga']),
                             round((len(temp_2chance[team]['2fgm']) + len(temp_2chance[team]['3fgm'])) /
                                   (len(temp_2chance[team]['2fgm']) +
                                    len(temp_2chance[team]['2fga']) +
                                    len(temp_2chance[team]['3fgm']) +
                                    len(temp_2chance[team]['3fga'])), 4), 
                             round((len(temp_2chance[team]['2fgm']) + 1.5 * len(temp_2chance[team]['3fgm'])) /
                                   (len(temp_2chance[team]['2fgm']) +
                                    len(temp_2chance[team]['2fga']) +
                                    len(temp_2chance[team]['3fgm']) +
                                    len(temp_2chance[team]['3fga'])), 4),
                             round((len(fgm2_split_team[team]) + 1.5 * len(fgm3_split_team[team])) /
                                   (len(fgm2_split_team[team]) + 
                                   len(fga2_split_team[team]) + 
                                   len(fgm3_split_team[team]) + 
                                   len(fga3_split_team[team]) ), 4),
                             
                            ((len(temp_2chance[team]['2fgm']) + 1.5 * len(temp_2chance[team]['3fgm'])) /
                                   (len(temp_2chance[team]['2fgm']) +
                                    len(temp_2chance[team]['2fga']) +
                                    len(temp_2chance[team]['3fgm']) +
                                    len(temp_2chance[team]['3fga'])) / 
                            ((len(fgm2_split_team[team]) + 1.5 * len(fgm3_split_team[team])) /
                                   (len(fgm2_split_team[team]) + 
                                   len(fga2_split_team[team]) + 
                                   len(fgm3_split_team[team]) + 
                                   len(fga3_split_team[team]) )))
                            ])
    
    
#eFG% = (2fgm + 1.5*3fgm) / (2fga + 3fga)

print(tabulate(reversed(sorted(temp_2change_efg, key=lambda x: x[9])), headers=('team', 'orebs', '+ 2fg', '- 2fg', '+ 3fg', '- 3fg', 'oreb-FG%','oreb-eFG%', 'eFG%', 'oreb-eFG / eFG')))

# Identify the top 5 teams & players whose offensive rebounds most effectively yield 2nd chance points.

Mostly, a good offensive team also scores efficiently on 2nd chance points, which doesn't say much about whether a team gets offensive rebounds that yield easier scoring opportunities. To discern the difference, the disparity between eFG% after an oreb, and eFG% overall for a team is measured. The sample sizes are too small and noisy to make strong claim.

__Zalgiris Kaunas (ZAL)__ is the first team among the top oreb-eFG%/eFG% ratio teams that don't suffer from unuseably small sample sizes. At 186 orebs, their oreb-eFG is 10% higher than overall-eFG. So to strictly answer the question, ZAL has to be included in the top 5 teams.

__Khimki Moscow Region (KHI)__ is the next up, their oreb-eFG just short of 10% higher than overall-eFG. Their sample size is also good, so one can claim that the 10% disparity is a product of the team's offensive rebounding dynamic, instead of statistical noise.

__CSKA Moscow (CSK)__ is not among the teams whose oreb-eFG numbers are way higher than their overal-eFG. However, they rebound more than both teams above on the list, (ZAL, KHI), and their oreb-eFG in general leads the league at almost 62%, the only team higher than 60%. But as stated above, their high oreb-eFG% is a product of their efficient offense in general.

__~~Dinamo Banco di Sardegna Sassari (SAS)~~__ has to be considered just because of their whopping 20% increase in eFG in 2nd chance points compared to overall eFG, and they get offensive rebounds at a pretty high clip, but they did not play many games, only 54 offensive rebounds in the season. Too noisy to count.

__Real Madrid (MAD)__ is 2nd in the league in raw offensive rebounds, and they get about 8% higher eFG after an oreb than overall, which is 6th best. Very solid all around, with a large sample size.

__Olympiacos Piraeus (OLY)__ is on the fringe, but sneaks in because of their large sample size, good overall oreb numbers, and 7th overall in oreb-eFG%/eFG% ratio. Nothing too flashy about them, but good enough.

In [ ]:
temp_2chance_player = []

temp_oreb_player={}
for p in player_infos:
    temp_oreb_player[p] = []

for o in oreb_fgs:
    if o['oreb']['player_id'] == '': continue
    temp_oreb_player[o['oreb']['player_id']].append(o['shot'])
    
for player_id, shots in temp_oreb_player.items():
    fga=(len([s for s in shots if any([s['event_desc_id'] == x for x in ['2FGM', '3FGM', '2FGA', '3FGA', '2FGAB', '3FGAB']])]))
    if fga==0: continue
    efg = (len([s for s in shots if s['event_desc_id'] == '2FGM']) + 1.5 * len([s for s in shots if s['event_desc_id'] == '3FGM'])) / fga
    if efg > 0.6 and fga > 20:
        temp_2chance_player.append((player_id, player_infos[player_id]['team'][0], efg, fga))


In [ ]:
print(tabulate(reversed(sorted(temp_2chance_player, key=lambda x: x[2]))))

## Top 5 Players whose offensive rebounds most effectively yield 2nd chance points

In [ ]:
for p in ['P002812', 'PJDU', 'P000925', 'P006503', 'PLHI', 'P005927']:
    print(player_infos[p]['name'])


These are the top 5 players in terms of eFG% of shots by him or his teammates after his offensive rebounds.

* ['HINES, KYLE']
* ['RADULJICA, MIROSLAV']
* ['VESELY, JAN']
* ['UDOH, EKPE']
* ~~['VORONTSEVICH, ANDREY']~~

I set the limit at 20 shot attempts after a player's offensive rebounds, and the results are decent. 'VORONTSEVICH, ANDREY' is very questionable because of his orebs -> shot attempts numbers at 25. I prefer ['AYON, GUSTAVO'], just because he puts up huge oreb numbers, and the subsequent shot attempt eFG% is 12th best. Not great, but extremely consistent.

My Final List

* ['HINES, KYLE']
* ['RADULJICA, MIROSLAV']
* ['VESELY, JAN']
* ['UDOH, EKPE']
* ['AYON, GUSTAVO']

In [ ]:
oreb_fgm2 = len([t for t in temp if t['event_desc_id'] == '2FGM'])
oreb_fga2 = len([t for t in temp if t['event_desc_id'] == '2FGA']) + len([t for t in temp if t['event_desc_id'] == '2FGAB'])
oreb_fgm3 = len([t for t in temp if t['event_desc_id'] == '3FGM'])
oreb_fga3 = len([t for t in temp if t['event_desc_id'] == '3FGA']) + len([t for t in temp if t['event_desc_id'] == '3FGAB'])
oreb_fgm = oreb_fgm2+oreb_fgm3
oreb_fga = oreb_fga2+oreb_fga3

print('fg% {} {}/{}'.format((round(oreb_fgm / (oreb_fgm + oreb_fga), 4)), oreb_fgm, (oreb_fgm+oreb_fga)))
print('2fg% {} {}/{}'.format((round(oreb_fgm2 / (oreb_fgm2 + oreb_fga2), 4)), oreb_fgm2, (oreb_fgm2+oreb_fga2)))
print('3fg% {} {}/{}'.format((round(oreb_fgm3 / (oreb_fgm3 + oreb_fga3), 4)), oreb_fgm3, (oreb_fgm3+oreb_fga3)))
print('efg% {} ({} + 1.5 * {}) / ({} + {})'.format(round(((oreb_fgm2 + 1.5 * oreb_fgm3)/(oreb_fgm + oreb_fga)), 4), oreb_fgm2, oreb_fgm3, oreb_fgm, oreb_fga))

In [ ]:
temp=[]
#temp.append(['overall', fgm, fgm+fga, round(100 * fgm/(fgm+fga), 2)])

print('All Shots')
print('-'*30)
temp.append(['FG', fgm, fgm+fga, round(100 * fgm/(fgm+fga), 2)])
temp.append(['2FG', fgm2, fgm2+fga2, round(100 * fgm2/(fgm2+fga2), 2)])
temp.append(['3FG', fgm3, fgm3+fga3, round(100 * fgm3/(fgm3+fga3), 2)])
temp.append(['eFG', None, None, round(100 * (fgm2 + 1.5 * fgm3)/(fgm+fga), 2)])
print(tabulate(temp, headers=('', 'made', 'attempted', 'FG%')))


# What is the 2P%, 3P%, eFG% of shots after offensive rebounds?

In [ ]:
temp=[]
#temp.append(['overall', fgm, fgm+fga, round(100 * fgm/(fgm+fga), 2)])

print('Shots after an Offensive Rebound')
print('-'*30)
temp.append(['FG', oreb_fgm, oreb_fgm+oreb_fga, round(100 * oreb_fgm/(oreb_fgm+oreb_fga), 2)])
temp.append(['2FG', oreb_fgm2, oreb_fgm2+oreb_fga2, round(100 * oreb_fgm2/(oreb_fgm2+oreb_fga2), 2)])
temp.append(['3FG', oreb_fgm3, oreb_fgm3+oreb_fga3, round(100 * oreb_fgm3/(oreb_fgm3+oreb_fga3), 2)])
temp.append(['eFG', None, None, round(100 * (oreb_fgm2 + 1.5 * oreb_fgm3)/(oreb_fgm+oreb_fga), 2)])
print(tabulate(temp, headers=('', 'made', 'attempted', 'FG%')))


In [ ]:
for r in pbp:
    if r['event_desc_id'] == '2FGA':
        print(r)
        break

# additional data would you acquire/use to predict likelihood of scoring 2nd chance points?

* location of orebs
* location of other players at the time of the missed shot and rebound (many defensive players may be out of position to defend another possession)
* height/position of the rebounder

From the EuroLeague 2014-15 and 2016-17 rulebooks

>Dimensions: The playing area must be adapted to the
dimensions and the markings established by FIBA. 

From the FIBA 2014-15 rulebooks, Article 2. Court

>__2.1. Playing court__

>The playing court shall have a flat, hard surface free from obstructions (Diagram 1)
with dimensions of 28 m in length by 15 m in width measured from the inner edge of
the boundary line.

>__2.2. Backcourt__

>A team's backcourt consists of its team's own basket, the inbounds part of the
backboard and that part of the playing court limited by the endline behind their own
basket, the sidelines and the centre line.

>__2.3. Frontcourt__

>A team's frontcourt consists of the opponents' basket, the inbounds part of the
backboard and that part of the playing court limited by the endline behind the
opponents' basket, the sidelines and the inner edge of the centre line nearest to the
opponents' basket.

>__2.4. Lines__

>All lines shall be drawn in white colour, 5 cm in width and clearly visible.

>>__2.4.1. Boundary line__

>>The playing court shall be limited by the boundary line, consisting of the endlines
and the sidelines. These lines are not part of the playing court.
Any obstruction including seated team bench personnel shall be at least 2 m from
the playing court.

>>__2.4.2. Centre line, centre circle and free-throw semi-circles__

>>The centre line shall be marked parallel to the endlines from the mid-point of the
sidelines. It shall extend 0.15 m beyond each sideline. The centre line is part of the
backcourt.
The centre circle shall be marked in the centre of the playing court and have a
radius of 1.80 m measured to the outer edge of the circumference. If the inside of
the centre circle is painted, it must be the same colour as the restricted areas.
The free-throw semi-circles shall be marked on the playing court with a radius of
1.80 m measured to the outer edge of the circumference and with their centres at
the mid-point of the free-throw lines (Diagram 2).

>>__2.4.3. Free-throw lines, restricted areas and free-throw rebound places__

>>The free-throw line shall be drawn parallel to each endline. It shall have its furthest
edge 5.80 m from the inner edge of the endline and shall be 3.60 m long. Its mid-point
shall lie on the imaginary line joining the mid-point of the 2 endlines.

>>The restricted areas shall be the rectangular areas marked on the playing court
limited by the endlines, the extended free-throw lines and the lines which originate
at the endlines, their outer edges being 2.45 m from the mid-point of the endlines
and terminating at the outer edge of the extended free-throw lines. These lines,
excluding the endlines, are part of the restricted area. The inside of the restricted
areas must be painted in one colour.

>>Free-throw rebound places along the restricted areas, reserved for players during
free throws, shall be marked as in Diagram 2.

>>__2.4.4. 3-point field goal area__

>>>The team's 3-point field goal area (Diagram 1 and Diagram 3) shall be the entire floor
area of the playing court, except for the area near the opponents' basket, limited by
and including:

>>>>• The 2 parallel lines extending from and perpendicular to the endline, with the
outer edge 0.90 m from the inner edge of the sidelines.

>>>>• An arc of radius 6.75 m measured from the point on the floor beneath the exact
centre of the opponents' basket to the outer edge of the arc. The distance of
the point on the floor from the inner edge of the mid-point of the endline is
1.575 m. The arc is joined to the parallel lines.
The 3-point line is not part of the 3-point field goal area.

From the 2015 amendments to the FIBA rulebook (which are the same as above word-for-word, except as marked)

>__2.4.3. Free-throw lines, restricted areas, and free-throw rebound places__

>The free-throw line shall be drawn parallel to each endline. It shall have its furthest edge 5.80 m from the inner edge of the endline and shall be 3.60 m long. Its mid-point shall lie on the imaginary line joining the mid-point of the two endlines.

>The restricted areas shall be the rectangular areas marked on the playing court limited by the endlines, the extended free-throw lines and the lines which originate at the endlines, their outer edges being 2.45 m from the mid-point of the endlines and terminating at the outer edge of the extended free-throw lines. These lines, excluding the endlines, are part of the restricted area. ~~The inside of the restricted areas must be painted in one colour.~~

>Free-throw rebound places along the restricted areas, reserved for players during free throws, shall be marked as in Diagram 2.

In [ ]:
Image('./img/fullcourt.jpg')

In [ ]:
Image('./img/paint.jpg')

In [ ]:
def getCoords(event):
    if event['coord_x'] == '' or event['coord_y'] == '': return None
    return (float(event['coord_x']) / 100.0, float(event['coord_y']) / 100.0)

In [ ]:
def getRegion(event):
    #Regions = ('2r', '2p', '2m', '3c', '3a')
    '''
    '''
    coords = getCoords(event)
    if any([event['event_desc_id'] == e for e in ('2FGA', '2FGAB', '2FGM')]):
        if abs(coords[0]) < 2.45 and coords[1] <  4.225:
            if (abs(coords[0]) < 1.5 and coords[1] > -0.375 and coords[1] <= 0.0) or (coords[1] > 0 and distance.euclidean((0,0), coords) < 1.25):
                return '2r'
            else: return '2p'
        else: return '2m'

    if any([event['event_desc_id'] == e for e in ('3FGA', '3FGAB', '3FGM')]):
        if coords[1] < 1.42:
            return '3c'
        else: return '3a'
    return None
        

In [ ]:
fgm = [m for team, m in madeshots_team.items()]
fga = [m for team, m in missedshots_team.items()]

temp=[]
for team in fgm:
    for shot in team:
        if shot['event_desc_id'] == '2FGM':
            temp.append(shot)
fgm2_event = temp

temp=[]
for team in fga:
    for shot in team:
        if shot['event_desc_id'] == '2FGA' or shot['event_desc_id'] == '2FGAB':
            temp.append(shot)
fga2_event = temp


temp=[]
for team in fgm:
    for shot in team:
        if shot['event_desc_id'] == '3FGM':
            temp.append(shot)
fgm3_event = temp

temp=[]
for team in fga:
    for shot in team:
        if shot['event_desc_id'] == '3FGA' or shot['event_desc_id'] == '3FGAB':
            temp.append(shot)
fga3_event = temp

In [ ]:
fg_region = {}
for region in ('2r', '2p', '2m', '3c', '3a'):
    fg_region[region] = ([o for o in fgm2_event if getRegion(o) == region] + 
                         [o for o in fgm3_event if getRegion(o) == region], 
                         [o for o in fga2_event if getRegion(o) == region] + 
                         [o for o in fga3_event if getRegion(o) == region])
    
for region, o in fg_region.items():
    print(region, len(o[0]), len(o[1]))

In [ ]:
oreb_fg

oreb_fgm2_event = [t for t in oreb_fg if t['event_desc_id'] == '2FGM']
oreb_fga2_event = [t for t in oreb_fg if t['event_desc_id'] == '2FGA'] + [t for t in oreb_fg if t['event_desc_id'] == '2FGAB']
oreb_fgm3_event = [t for t in oreb_fg if t['event_desc_id'] == '3FGM']
oreb_fga3_event = [t for t in oreb_fg if t['event_desc_id'] == '3FGA'] + [t for t in oreb_fg if t['event_desc_id'] == '3FGAB']

print(len(oreb_fgm2_event))
print(len(oreb_fga2_event))

print(len(oreb_fgm3_event))
print(len(oreb_fga3_event))

In [ ]:
oreb_fg_region = {}
for region in ('2r', '2p', '2m', '3c', '3a'):
    oreb_fg_region[region] = ([o for o in oreb_fgm2_event if getRegion(o) == region] + 
                              [o for o in oreb_fgm3_event if getRegion(o) == region], 
                              [o for o in oreb_fga2_event if getRegion(o) == region] + 
                              [o for o in oreb_fga3_event if getRegion(o) == region])
    
for region, o in oreb_fg_region.items():
    print(region, len(o[0]), len(o[1]))
  

In [ ]:
temp=[]
fg_region_tab={}
for region, o in fg_region.items():
    fg_region_tab[region] = [len(o[0]), len(o[0])+len(o[1]), round(len(o[0]) / (len(o[0])+len(o[1])),4)]
    temp.append((region, len(o[0]), len(o[0])+len(o[1]), round(len(o[0]) / (len(o[0])+len(o[1])),4) ))
    
print(tabulate(temp, headers=('region', 'made', 'attempted', 'fg%')))

# (Bonus) What is the FG% of shots after offensive rebounds in each court zone (restricted area, (non-RA) in-the-paint, midrange, corner 3, above break 3)?

In [ ]:
  
temp=[]
oreb_fg_region_tab={}
for region, o in oreb_fg_region.items():
    oreb_fg_region_tab[region] = [len(o[0]), len(o[0])+len(o[1]), round(len(o[0]) / (len(o[0])+len(o[1])),4)]
    temp.append((region, len(o[0]), len(o[0])+len(o[1]), round(len(o[0]) / (len(o[0])+len(o[1])),4) ))

print(tabulate(temp, headers=('region', 'made', 'attempted', 'fg%')))

In [ ]:
made = [(o['coord_x'], o['coord_y']) for o in oreb_fgm2_event] + [(o['coord_x'], o['coord_y']) for o in oreb_fgm3_event]

missed = [(o['coord_x'], o['coord_y']) for o in oreb_fga2_event] + [(o['coord_x'], o['coord_y']) for o in oreb_fga3_event]
print(len(made), len(missed))

(x,y) coordinates for shots have already been cleaned up, folding the locations into one basket.

In [ ]:

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)

lines=[]
#out of bounds
lines.append(Line2D([-7.5],[-1.575, 14], color='k'))
lines.append(Line2D([7.5], [-1.575, 14], color='k'))
lines.append(Line2D([-7.5, 7.5], [-1.575], color='k'))
#center line
lines.append(Line2D([-7.5, 7.5], [14], color='k'))

#paint
lines.append(Line2D([-2.45, 2.45], [4.225], color='k'))
lines.append(Line2D([-2.45], [-1.575, 4.225], color='k'))
lines.append(Line2D([2.45], [-1.575, 4.225], color='k'))

#backboard
lines.append(Line2D([-.5, .5], [-.375], color='k'))

#restricted area extend
lines.append(Line2D([-1.25], [0, -0.375], color='k'))
lines.append(Line2D([1.25], [0, -0.375], color='k'))

#3 point line extended (corner 3)
lines.append(Line2D([-7.5+0.9], [-1.575, 1.42], color='k'))
lines.append(Line2D([7.5-0.9], [-1.575, 1.42], color='k'))
             
for l in lines:
    ax.add_line(l)

circles=[]
#hoop
circles.append(plt.Circle((0, 0), 0.25, facecolor='none', edgecolor='k'))
#(0,0) dot
circles.append(plt.Circle((0, 0), 0.01, facecolor='none', edgecolor='k'))
#restricted area
#circles.append(plt.Circle((0, 0), 1.25, facecolor='none', edgecolor='k'))

for c in circles:
    ax.add_artist(c)
    
#w1 = Wedge((0,0), 1.25, 0, 180, facecolor='none', edgecolor='k')
#ax.add_artist(w1)

arcs=[]
#restricted area
arcs.append(mpatches.Arc((0,0), width = 2.5, height = 2.5, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#free throw half-circle
arcs.append(mpatches.Arc((0,4.225), width = 3.6, height = 3.6, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#3 point line
arcs.append(mpatches.Arc((0,0), width = 13.5, height = 13.5, theta1=12.1, theta2=180.0-12.1, color='black', linewidth=1.5))
#center circle
arcs.append(mpatches.Arc((0, 14), width = 3.6, height = 3.6, theta1=180, theta2=0, color='black', linewidth=1.5))

for a in arcs:
    ax.add_patch(a)

# missed_all=[]
# made_all=[]
# for team, events in madeshots_team.items():
#     for event in events:
#         made_all.append((event['coord_x'], event['coord_y']))
        
# for team, events in missedshots_team.items():
#     for event in events:
#         missed_all.append((event['coord_x'], event['coord_y']))

# scats.append(ax.scatter([float(m[0])/100.0 for m in missed_all], [float(m[1])/100.0 for m in missed_all], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made_all], [float(m[1])/100.0 for m in made_all], 10, edgecolor='none', facecolor='g'))

# scats=[]
# scats.append(ax.scatter([float(m[0])/100.0 for m in missed], [float(m[1])/100.0 for m in missed], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made], [float(m[1])/100.0 for m in made], 10, edgecolor='none', facecolor='g'))

scats = []
#regions = ('2p', '2r', '2m', '3c', '3a')
regions = ['2p']
scats.append(ax.scatter([float(o['coord_x'])/100.0 for o in oreb_fga2_event if any([getRegion(o) == e for e in regions])] + 
                        [float(o['coord_x'])/100.0 for o in oreb_fga3_event if any([getRegion(o) == e for e in regions])], 
                        [float(o['coord_y'])/100.0 for o in oreb_fga2_event if any([getRegion(o) == e for e in regions])] + 
                        [float(o['coord_y'])/100.0 for o in oreb_fga3_event if any([getRegion(o) == e for e in regions])], 
                        20, marker = 'x', edgecolor='none', facecolor='r', alpha=0.5))
scats.append(ax.scatter([float(o['coord_x'])/100.0 for o in oreb_fgm2_event if any([getRegion(o) == e for e in regions])] + 
                        [float(o['coord_x'])/100.0 for o in oreb_fgm3_event if any([getRegion(o) == e for e in regions])], 
                        [float(o['coord_y'])/100.0 for o in oreb_fgm2_event if any([getRegion(o) == e for e in regions])] + 
                        [float(o['coord_y'])/100.0 for o in oreb_fgm3_event if any([getRegion(o) == e for e in regions])], 
                        30, marker = 'o', edgecolor='g', facecolor='none', alpha=0.6))

# plt.legend(scats, ('made', 'missed'), loc = 'upper right')



ax.set_xlim(-8, 8)
ax.set_ylim(-2.075, 10)
ax.set_aspect('equal')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('')

plt.show()



In [ ]:

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)

lines=[]
#out of bounds
lines.append(Line2D([-7.5],[-1.575, 14], color='k'))
lines.append(Line2D([7.5], [-1.575, 14], color='k'))
lines.append(Line2D([-7.5, 7.5], [-1.575], color='k'))
#center line
lines.append(Line2D([-7.5, 7.5], [14], color='k'))

#paint
lines.append(Line2D([-2.45, 2.45], [4.225], color='k'))
lines.append(Line2D([-2.45], [-1.575, 4.225], color='k'))
lines.append(Line2D([2.45], [-1.575, 4.225], color='k'))

#backboard
lines.append(Line2D([-.5, .5], [-.375], color='k'))

#restricted area extend
lines.append(Line2D([-1.25], [0, -0.375], color='k'))
lines.append(Line2D([1.25], [0, -0.375], color='k'))

#3 point line extended (corner 3)
lines.append(Line2D([-7.5+0.9], [-1.575, 1.42], color='k'))
lines.append(Line2D([7.5-0.9], [-1.575, 1.42], color='k'))

#imaginary lines for zones

             
for l in lines:
    ax.add_line(l)

circles=[]
#hoop
circles.append(plt.Circle((0, 0), 0.25, facecolor='none', edgecolor='k'))
#(0,0) dot
circles.append(plt.Circle((0, 0), 0.01, facecolor='none', edgecolor='k'))
#restricted area
#circles.append(plt.Circle((0, 0), 1.25, facecolor='none', edgecolor='k'))

for c in circles:
    ax.add_artist(c)
    
#w1 = Wedge((0,0), 1.25, 0, 180, facecolor='none', edgecolor='k')
#ax.add_artist(w1)

arcs=[]
#restricted area
arcs.append(mpatches.Arc((0,0), width = 2.5, height = 2.5, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#free throw half-circle
arcs.append(mpatches.Arc((0,4.225), width = 3.6, height = 3.6, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#3 point line
arcs.append(mpatches.Arc((0,0), width = 13.5, height = 13.5, theta1=12.1, theta2=180.0-12.1, color='black', linewidth=1.5))
#center circle
arcs.append(mpatches.Arc((0, 14), width = 3.6, height = 3.6, theta1=180, theta2=0, color='black', linewidth=1.5))

for a in arcs:
    ax.add_patch(a)

# missed_all=[]
# made_all=[]
# for team, events in madeshots_team.items():
#     for event in events:
#         made_all.append((event['coord_x'], event['coord_y']))
        
# for team, events in missedshots_team.items():
#     for event in events:
#         missed_all.append((event['coord_x'], event['coord_y']))

# scats.append(ax.scatter([float(m[0])/100.0 for m in missed_all], [float(m[1])/100.0 for m in missed_all], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made_all], [float(m[1])/100.0 for m in made_all], 10, edgecolor='none', facecolor='g'))

# scats=[]
# scats.append(ax.scatter([float(m[0])/100.0 for m in missed], [float(m[1])/100.0 for m in missed], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made], [float(m[1])/100.0 for m in made], 10, edgecolor='none', facecolor='g'))

# plt.legend(scats, ('made', 'missed'), loc = 'upper right')

#fg% by zone


#fill_between
#so3
x = np.arange(-7.5, 7.5,0.01)
line_so3=10
line_so3_a=5
plt.fill_between(x, line_so3, line_so3_a, color='g', alpha=1)
#rw3
x = np.arange(-7.5, 0,0.01)
line_rw3=-1.7245*x
plt.fill_between(x, line_rw3, color='grey', alpha=1)
#lw3
x = np.arange(0, 7.5,0.01)
line_lw3=1.7245*x
plt.fill_between(x, line_lw3, color='grey', alpha=1)
#rw2
x = np.arange(-7.5+0.9, 0,0.01)
line_rw2=np.sqrt(6.75**2 - x**2)
line_rw2_a=1.42
plt.fill_between(x, line_rw2, line_rw2_a, color='y', alpha=1)
#lw2
x = np.arange(0, 7.5-0.9,0.01)
line_rw2=np.sqrt(6.75**2 - x**2)
line_rw2_a=1.42
plt.fill_between(x, line_rw2, line_rw2_a, color='orange', alpha=1)
#rc3
x = np.arange(-7.5, -7.5+0.9,0.01)
line_rc3=1.42
line_rc3_a=-1.575
plt.fill_between(x, line_rc3, line_rc3_a, color='red', alpha=1)
#lc3
x = np.arange(7.5-0.9, 7.5,0.01)
line_lc3=1.42
line_lc3_a=-1.575
plt.fill_between(x, line_lc3, line_lc3_a, color='red', alpha=1)
#rc2
x = np.arange(-7.5+0.9, -2.45, 0.01)
line_rc2=1.42
line_rc2_a=-1.575
plt.fill_between(x, line_rc2, line_rc2_a, color='g', alpha=1)
#lc2
x = np.arange(2.45, 7.5-0.9, 0.01)
line_lc2=1.42
line_lc2_a=-1.575
plt.fill_between(x, line_lc2, line_lc2_a, color='g', alpha=1)
#pt2
x = np.arange(-2.45, 2.45, 0.01)
line_lc2=4.225
line_lc2_a=-1.575
plt.fill_between(x, line_lc2, line_lc2_a, color='b', alpha=1)
#so2
x = np.arange(-2.45, 2.45, 0.01)
line_rw2=np.sqrt(6.75**2 - x**2)
line_rw2_a=4.225
plt.fill_between(x, line_rw2, line_rw2_a, color='r', alpha=1)
#ra2
x = np.arange(-1.25, 1.25, 0.01)
line_ra2=np.sqrt(1.25**2 - x**2)
line_ra2_a=-0.375
plt.fill_between(x, line_ra2, line_ra2_a, color='grey', alpha=1)

ax.set_xlim(-8, 8)
ax.set_ylim(-2.075, 10)
ax.set_aspect('equal')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('')

plt.show()



# (Bonus) Build a visualization to show FG% of shots after offensive rebounds.


In [ ]:

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)

lines=[]
#out of bounds
lines.append(Line2D([-7.5],[-1.575, 14], color='k'))
lines.append(Line2D([7.5], [-1.575, 14], color='k'))
lines.append(Line2D([-7.5, 7.5], [-1.575], color='k'))
#center line
lines.append(Line2D([-7.5, 7.5], [14], color='k'))

#paint
lines.append(Line2D([-2.45, 2.45], [4.225], color='k'))
lines.append(Line2D([-2.45], [-1.575, 4.225], color='k'))
lines.append(Line2D([2.45], [-1.575, 4.225], color='k'))

#backboard
lines.append(Line2D([-.5, .5], [-.375], color='k'))

#restricted area extend
lines.append(Line2D([-1.25], [0, -0.375], color='k'))
lines.append(Line2D([1.25], [0, -0.375], color='k'))

#3 point line extended (corner 3)
lines.append(Line2D([-7.5+0.9], [-1.575, 1.42], color='k'))
lines.append(Line2D([7.5-0.9], [-1.575, 1.42], color='k'))

#imaginary lines for zones

             
for l in lines:
    ax.add_line(l)

circles=[]
#hoop
circles.append(plt.Circle((0, 0), 0.25, facecolor='none', edgecolor='k'))
#(0,0) dot
circles.append(plt.Circle((0, 0), 0.01, facecolor='none', edgecolor='k'))
#restricted area
#circles.append(plt.Circle((0, 0), 1.25, facecolor='none', edgecolor='k'))

for c in circles:
    ax.add_artist(c)
    
#w1 = Wedge((0,0), 1.25, 0, 180, facecolor='none', edgecolor='k')
#ax.add_artist(w1)

arcs=[]
#restricted area
arcs.append(mpatches.Arc((0,0), width = 2.5, height = 2.5, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#free throw half-circle
arcs.append(mpatches.Arc((0,4.225), width = 3.6, height = 3.6, theta1=0.0, theta2=180.0, color='black', linewidth=1.5))
#3 point line
arcs.append(mpatches.Arc((0,0), width = 13.5, height = 13.5, theta1=12.1, theta2=180.0-12.1, color='black', linewidth=1.5))
#center circle
arcs.append(mpatches.Arc((0, 14), width = 3.6, height = 3.6, theta1=180, theta2=0, color='black', linewidth=1.5))

for a in arcs:
    ax.add_patch(a)

# missed_all=[]
# made_all=[]
# for team, events in madeshots_team.items():
#     for event in events:
#         made_all.append((event['coord_x'], event['coord_y']))
        
# for team, events in missedshots_team.items():
#     for event in events:
#         missed_all.append((event['coord_x'], event['coord_y']))

# scats.append(ax.scatter([float(m[0])/100.0 for m in missed_all], [float(m[1])/100.0 for m in missed_all], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made_all], [float(m[1])/100.0 for m in made_all], 10, edgecolor='none', facecolor='g'))

# scats=[]
# scats.append(ax.scatter([float(m[0])/100.0 for m in missed], [float(m[1])/100.0 for m in missed], 10, edgecolor='none', facecolor='r'))
# scats.append(ax.scatter([float(m[0])/100.0 for m in made], [float(m[1])/100.0 for m in made], 10, edgecolor='none', facecolor='g'))

# plt.legend(scats, ('made', 'missed'), loc = 'upper right')

#fg% by zone -> heatmap
oreb_fg_region_tab
norm = matplotlib.colors.Normalize(vmin=-0.04,vmax=0.04)
heatmap=cm.get_cmap('RdYlGn')
getcolor = lambda x: heatmap(norm(x), alpha=1)

#regions = ('2r', '2p', '2m', '3c', '3a')
#fill_between
#so3
x = np.arange(-7.5, 7.5,0.01)
line_a3=1.42
line_a3_a=10
plt.fill_between(x, line_a3_a, line_a3, color=getcolor(oreb_fg_region_tab['3a'][2] - fg_region_tab['3a'][2]))
# #rw3
# x = np.arange(-7.5, 0,0.01)
# line_rw3=-1.7245*x
# plt.fill_between(x, line_rw3, color='grey', alpha=1)
#lw3
x = np.arange(-7.5+0.9, 7.5-0.9,0.01)
line_m2=np.sqrt(6.75**2 - x**2)
line_m2_a=-1.575
plt.fill_between(x, line_m2, line_m2_a, color=getcolor(oreb_fg_region_tab['2m'][2] - fg_region_tab['2m'][2]))
# #rw2
# x = np.arange(-7.5+0.9, 0,0.01)
# line_rw2=np.sqrt(6.75**2 - x**2)
# line_rw2_a=1.42
# plt.fill_between(x, line_rw2, line_rw2_a, color='y', alpha=1)
# #lw2
# x = np.arange(0, 7.5-0.9,0.01)
# line_rw2=np.sqrt(6.75**2 - x**2)
# line_rw2_a=1.42
# plt.fill_between(x, line_rw2, line_rw2_a, color='orange', alpha=1)
#rc3
x = np.arange(-7.5, -7.5+0.9,0.01)
line_rc3=1.42
line_rc3_a=-1.575
plt.fill_between(x, line_rc3, line_rc3_a, color=getcolor(oreb_fg_region_tab['3c'][2] - fg_region_tab['3c'][2]))
#lc3
x = np.arange(7.5-0.9, 7.5,0.01)
line_lc3=1.42
line_lc3_a=-1.575
plt.fill_between(x, line_lc3, line_lc3_a, color=getcolor(oreb_fg_region_tab['3c'][2] - fg_region_tab['3c'][2]))
# #rc2
# x = np.arange(-7.5+0.9, -2.45, 0.01)
# line_rc2=1.42
# line_rc2_a=-1.575
# plt.fill_between(x, line_rc2, line_rc2_a, color='g', alpha=1)
# #lc2
# x = np.arange(2.45, 7.5-0.9, 0.01)
# line_lc2=1.42
# line_lc2_a=-1.575
# plt.fill_between(x, line_lc2, line_lc2_a, color='g', alpha=1)
#pt2
x = np.arange(-2.45, 2.45, 0.01)
line_lc2=4.225
line_lc2_a=-1.575
plt.fill_between(x, line_lc2, line_lc2_a, color=getcolor(oreb_fg_region_tab['2p'][2] - fg_region_tab['2p'][2]))
# #so2
# x = np.arange(-2.45, 2.45, 0.01)
# line_rw2=np.sqrt(6.75**2 - x**2)
# line_rw2_a=4.225
# plt.fill_between(x, line_rw2, line_rw2_a, color='r', alpha=1)
#ra2
x = np.arange(-1.25, 1.25, 0.01)
line_ra2=np.sqrt(1.25**2 - x**2)
line_ra2_a=-0.375
plt.fill_between(x, line_ra2, line_ra2_a, color=getcolor(oreb_fg_region_tab['2r'][2] - fg_region_tab['2r'][2]))

ax1 = fig.add_axes([0.05, 0.15, 0.9, 0.025])

cmap = matplotlib.cm.RdYlGn
norm = matplotlib.colors.Normalize(vmin=-0.04, vmax=0.04)

cb1 = matplotlib.colorbar.ColorbarBase(ax1, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

ax.set_xlim(-8, 8)
ax.set_ylim(-2.075, 10)
ax.set_aspect('equal')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('FG% difference: after oreb - overall')

plt.show()



In [ ]:
regions = ['3c', '3a', '2m', '2p', '2r']
for r in regions:
    print(r, oreb_fg_region_tab[r][2] - fg_region_tab[r][2])